In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.jmt.*
import jline.solvers.ssa.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
import jline.solvers.nc.*
import jline.solvers.mam.*
import jline.solvers.*
import jline.util.matrix.*

In [ ]:
val model = Network("model")

// Create nodes
val delay = Delay(model, "Delay")
val queue1 = Queue(model, "Queue1", SchedStrategy.PS)

// Create two closed job classes with equal populations
val jobclass1 = ClosedClass(model, "Class1", 2, delay, 0)  // 2 jobs
val jobclass2 = ClosedClass(model, "Class2", 2, delay, 0)  // 2 jobs

In [ ]:
// Set service processes with different distributions
// Delay station service times
delay.setService(jobclass1, Erlang(3, 2))                    // Erlang(3,2) - lower variability
delay.setService(jobclass2, HyperExp(0.5, 3.0, 10.0))       // HyperExp(0.5,3,10) - high variability

// Queue1 station service times
queue1.setService(jobclass1, HyperExp(0.1, 1.0, 10.0))      // HyperExp(0.1,1,10) - high variability
queue1.setService(jobclass2, Exp(1.0))                      // Exp(1) - standard exponential

In [ ]:
// Set routing matrix with class switching
val P = model.initRoutingMatrix()

// Class1 routing patterns
// P{1,1} = [0.3,0.1; 0.2,0]
P.set(jobclass1, jobclass1, delay, delay, 0.3)   // Stay at Delay (30%)
P.set(jobclass1, jobclass1, delay, queue1, 0.1)  // Delay -> Queue1 (10%)
P.set(jobclass1, jobclass1, queue1, delay, 0.2)  // Queue1 -> Delay (20%)
P.set(jobclass1, jobclass1, queue1, queue1, 0.0)

// Class1 -> Class2 switching
// P{1,2} = [0.6,0.0; 0.8,0]
P.set(jobclass1, jobclass2, delay, delay, 0.6)   // Class1 -> Class2 at Delay (60%)
P.set(jobclass1, jobclass2, delay, queue1, 0.0)
P.set(jobclass1, jobclass2, queue1, delay, 0.8)  // Class1 -> Class2 at Queue1 -> Delay (80%)
P.set(jobclass1, jobclass2, queue1, queue1, 0.0)

// Class2 -> Class1 switching
// P{2,1} = [0,0; 1,0]
P.set(jobclass2, jobclass1, delay, delay, 0.0)
P.set(jobclass2, jobclass1, delay, queue1, 0.0)
P.set(jobclass2, jobclass1, queue1, delay, 1.0)  // Class2 -> Class1 at Queue1 -> Delay (100%)
P.set(jobclass2, jobclass1, queue1, queue1, 0.0)

// Class2 routing
// P{2,2} = [0,1; 0,0]
P.set(jobclass2, jobclass2, delay, delay, 0.0)
P.set(jobclass2, jobclass2, delay, queue1, 1.0)  // Delay -> Queue1 (100%)
P.set(jobclass2, jobclass2, queue1, delay, 0.0)
P.set(jobclass2, jobclass2, queue1, queue1, 0.0)

model.link(P)

In [ ]:
// Create array of different solvers to compareval solvers = arrayOf(    CTMC(model),    JMT(model, "seed", 23000, "verbose", true, "samples", 5000),    SSA(model, "seed", 23000, "verbose", true, "samples", 5000),    FLD(model),    MVA(model, "method", "exact"),    NC(model, "method", "exact"),    MAM(model),    LINE(model))// Execute each solver and display resultsfor (solver in solvers) {        try {        val avgTable = solver.getAvgTable()        avgTable.print()    } catch (e: Exception) {        println("Error with ${solver.getName()}: ${e.message}")    }}

This two-class model with class switching demonstrates:

1. **Job Class Characteristics**:
   - **Class1**: Erlang service at delay (low variability), HyperExp at queue (high variability)
   - **Class2**: HyperExp service at delay (high variability), Exp at queue (standard)

2. **Class Switching Patterns**:
   - Class1 can switch to Class2 (60% at delay, 80% after queue service)
   - Class2 always switches back to Class1 after queue service
   - This creates a dynamic population exchange

3. **Service Distribution Variety**:
   - **Erlang**: Lower variability (more predictable service)
   - **Hyperexponential**: High variability (mix of fast and slow service)
   - **Exponential**: Standard memoryless service

4. **Routing Complexity**:
   - Probabilistic routing within Class1
   - Deterministic routing for Class2
   - Class switching creates complex dynamics

5. **Multiple Solution Methods**:
   - CTMC: Exact Markov chain analysis
   - JMT/SSA: Simulation-based approaches
   - Fluid: Continuous approximation
   - MVA/NC: Analytical queueing methods
   - MAM: Matrix analytical methods

The model represents systems where:
- Jobs change characteristics during processing
- Different service variabilities affect performance
- Class switching models priority changes or job transformations
- Multiple validation methods ensure solution accuracy